In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
# inline plotting
%matplotlib inline

In [3]:
from labproject.experiments import Experiment, ScaleDim, ScaleSampleSize
from labproject.utils import get_log_path
from labproject.plotting import generate_palette, color_dict

from labproject.data import DATASETS, DISTRIBUTIONS
from labproject.metrics import METRICS
from torch.distributions import MultivariateNormal

In [4]:
from labproject.data import DATASETS, DISTRIBUTIONS
from labproject.utils import get_cfg, set_seed, get_log_path
from labproject.data import get_dataset
import os
from omegaconf import OmegaConf
from labproject.experiments import *
from labproject.plotting import cm2inch
import time
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal

In [5]:
"""dict = color_dict
print(f"Dict: {dict}")"""

"""col_map = {
    'ScaleSampleSize': 'XX',
    'ScaleSampleSizeKL': 'XX',
    'ScaleSampleSizeSW': 'wasserstein',
    'ScaleSampleSizeMMD': 'mmd',
    'ScaleSampleSizeC2ST': 'c2st',
    'ScaleDim': 'XX',
    'ScaleDimKL': 'XX',
    'ScaleDimSW': 'wasserstein',
    'ScaleDimMMD': 'mmd',
    'ScaleDimC2ST': 'c2st',
}"""

from labproject.plotting import generate_palette
color_dict = {"wasserstein": "#cc241d",
              "mmd": "#eebd35",
              "c2st": "#458588",
              "fid": "#8ec07c", 
              "kl": "#8ec07c"}


col_map = {'ScaleDimKL':'kl', 'ScaleDimSW':'wasserstein',
           'ScaleDimMMD':'mmd', 'ScaleDimC2ST':'c2st',
           'ScaleDimFID':'fid', }



In [6]:
METRICS.keys()


dict_keys(['mmd_rbf', 'mmd_rbf_median_heuristic', 'mmd_rbf_auto', 'mmd_polynomial', 'mmd_linear_naive', 'mmd_linear', 'c2st_nn', 'c2st_rf', 'c2st_knn', 'gaussian_kl_divergence', 'wasserstein_gauss_squared', 'sliced_wasserstein', 'wasserstein_kuhn', 'wasserstein_sinkhorn'])

In [7]:
DATASETS.keys()

dict_keys(['random', 'multivariate_normal', 'toy_2d', 'cifar10_train', 'cifar10_test', 'imagenet_real_embeddings', 'imagenet_uncond_embeddings', 'imagenet_unconditional_model_embedding', 'imagenet_test_embedding', 'imagenet_validation_embedding', 'imagenet_conditional_model'])

In [8]:
print("Running experiments...")
# load the config file
cfg = OmegaConf.load("../../configs/conf_dims.yaml")
cfg.running_user = 'scaling_dims'
seed = cfg.seed

set_seed(seed)
print(f"Seed: {seed}")
print(f"Experiments: {cfg.experiments}") 
print(f"Data: {cfg.data}\n")

col_dark = {}
col_light = {}
for e, exp_name in enumerate(cfg.experiments):
    col_dark[exp_name] = generate_palette(color_dict[col_map[exp_name]], saturation='dark')[2]
    col_light[exp_name] = generate_palette(color_dict[col_map[exp_name]], saturation='light')[-1]
color_list = [col_light, col_dark] # make this a list to account for true and shifted
 
mapping = {'ScaleDimKL':'KL', 'ScaleDimSW':'SW',
        'ScaleDimMMD':'MMD', 'ScaleDimC2ST':'C2ST',
        'ScaleDimFID':'FD', }

# assert cfg.data is list
assert len(cfg.data) == len(cfg.n) == len(cfg.d), "Data, n and d must be lists of the same length"
    
dataset_fn = get_dataset(name=cfg.data[0])

# make comparison plots
fig, axes = plt.subplots(len(cfg.experiments), len(cfg.distort), figsize=cm2inch((20, 13)), sharex='col')
for ax in axes.flatten():
    # move spines outward
    ax.spines['bottom'].set_position(('outward', 4))
    ax.spines['left'].set_position(('outward', 4))
    ax.locator_params(nbins=4)

# Base datasets
dataset_gt = dataset_fn(cfg.n[0]*cfg.runs, cfg.d[0], distort=None)
dataset_intra = dataset_fn(cfg.n[0]*cfg.runs, cfg.d[0], distort=None)
print(f"Dataset shape: {dataset_gt.shape}")
print(f"Dataset shape: {dataset_intra.shape}")

for d, distort in enumerate(cfg.distort):
    print(f"Distort: {distort.upper()}")
    label_true = {}
    label_shift = {}
    for e, data_name in enumerate(cfg.data):
        label_true[data_name] = "true"
        if distort == 'shift_one' or distort=='shift_all':
            label_shift[data_name] = 'shifted'
        if distort == 'increase_var':
            label_shift[data_name] = 'increased var'
    label_list = [label_true, label_shift]

    # Distorted 
    dataset_inter = dataset_fn(cfg.n[0]*cfg.runs, cfg.d[0], distort=cfg.distort[d])
    dataset_inter_var = dataset_fn(cfg.n[0]*cfg.runs, cfg.d[0], distort=cfg.distort[d])

    # Data plots
    fig_data, axes_data = plt.subplots(1, 4, figsize=cm2inch((15, 4)), sharex=True, sharey=True)
    axes_data[0].plot(dataset_gt[:1000, 0], dataset_gt[:1000, 1], 'o', label='gt', alpha=0.2, color='C0')
    axes_data[1].plot(dataset_intra[:1000, 0], dataset_intra[:1000, 1], 'o', label='true',  alpha=0.2, color='C2')
    axes_data[2].plot(dataset_inter[:1000, 0], dataset_inter[:1000, 1], 'o', label=cfg.distort[d] ,alpha=0.2, color='C3')
    axes_data[3].plot(dataset_inter_var[:1000, 0], dataset_inter[:1000, 1], 'o', label=cfg.distort[d] ,alpha=0.2, color='C4')
    for ax in axes_data:
        ax.legend()

    for e, exp_name in enumerate(cfg.experiments):
        experiment = globals()[exp_name]()
        time_start = time.time()
        dataset1 = dataset_gt
        ax = axes[e, d]
        ax.set_xscale('log')
        for dc, data_comp in enumerate([dataset_intra, dataset_inter]):
            dataset2 = data_comp
            assert dataset1.shape == dataset2.shape, f"Dataset shapes do not match: {dataset1.shape} vs. {dataset2.shape}"
            if mapping[exp_name]  == 'MMD':
                print(f'MMD {cfg.data} {d} {cfg.mmd_bandwidth[d]}')
                output = experiment.run_experiment(dataset1=dataset1,
                                                dataset2=dataset2,
                                                dataset_size = cfg.n[0],
                                                dim_sizes=cfg.dim_sizes,
                                                nb_runs = cfg.runs,
                                                bandwidth=cfg.mmd_bandwidth[d])
            else:
                output = experiment.run_experiment(dataset1=dataset1,
                                                    dataset2=dataset2,
                                                    dataset_size = cfg.n[0],
                                                    dim_sizes=cfg.dim_sizes,
                                                    nb_runs = cfg.runs)
            time_end = time.time()
            print(f"Experiment {exp_name} finished in {time_end - time_start}")

            log_path = get_log_path(cfg)
            os.makedirs(os.path.dirname(log_path), exist_ok=True)
            experiment.log_results(output, log_path)
            print(f"Numerical results saved to {log_path}")
            experiment.plot_experiment(*output,
                cfg.data[0],
                ax=ax,
                color=color_list[dc][exp_name],
                label=label_list[dc][cfg.data[0]],
                linestyle='-' if dc == 0 else '--',lw=2, marker='o'
            )
            if mapping[exp_name]  == 'MMD':
                ax.set_ylabel(mapping[exp_name]+ str(int(cfg.mmd_bandwidth[d])))
            else:
                ax.set_ylabel(mapping[exp_name])
            ax.set_xlabel('')
            if mapping[exp_name]  == 'C2ST':
                ax.set_ylim([0.45,1.1])
                ax.set_yticks([0.5,1.1])
            
        ax.legend()

axes[-1, 0].set_xlabel('dimensions')
axes[-1, 1].set_xlabel('dimensions')
axes[-1, 2].set_xlabel('dimensions')

fig.tight_layout()
print(f"Plots saved to {cfg.data[0]}.png")
fig.savefig(f"../../plots/metric_comparison_sim_budget_{cfg.n[0]}_dim_size_{cfg.data[0]}_NEW.png", dpi=300)
fig.savefig(f"../../plots/metric_comparison_sim_budget_{cfg.n[0]}_dim_size_{cfg.data[0]}_NEW.pdf", dpi=300)

print("Finished running experiments.")


Running experiments...
Seed: 1404
Experiments: ['ScaleDimSW', 'ScaleDimC2ST', 'ScaleDimMMD']
Data: ['multivariate_normal']

